# **Level 1**

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
from pyspark.sql import SparkSession
#from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
pet_products_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
pet_products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = pet_products_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789| 2015-08-31|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483| 2015-08-31|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658| 2015-08-31|
|R33GITXNUF1AD4|   33930128|B00BOEXWFG|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|   43534290|B001HBBQKY|     420905252| 2015-08

In [ ]:
 products_df = pet_products_df.select(["product_id", "product_title"]).drop_duplicates()
 products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
reviews_df = pet_products_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show(10)

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
| REAKC26P07MDN|A great purchase ...|Best belly bands ...|
|R3NU7OMZ4HQIEG|My dogs love Hugs...|My dogs love Hugs...|
|R14QJW3XF8QO1P|I have been purch...|I have been purch...|
|R2HB7AX0394ZGY|it is easy to ope...|It is extremely w...|
| RGKMPDQGSAHR3|           Dog crate|Worked really wel...|
|R1DJCVPQGCV66E|          Five Stars|I love my gates! ...|
|R3V52EAWLPBFQG|Seat belt tugs on...|Didn't quite work...|
|R3DKO8J1J28QBI|Great Pole, but S...|I had the origina...|
| R764DBXGRNECG|     My cat loves it|The pad is very s...|
| RW1853GAT0Z9F|          Five Stars|My cat drinks mor...|
+--------------+--------------------+--------------------+
only showing top 10 rows



In [ ]:
customers_df = pet_products_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine_df = pet_products_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



Push to AWS RDS instance

In [ ]:
from getpass import getpass
username = getpass('Enter database username')
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/bigdata_Postgres_db"
config = {"user":username, "password": password, "driver":"org.postgresql.Driver"}

Enter database username··········
Enter database password··········


In [ ]:
 # Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
 # Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
 # Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# **Level 2**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-2").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
pet_products_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
pet_products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

**Filter by votes**

In [5]:
 filtered_pet_products_df = pet_products_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_pet_products_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          2|            0|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            0|          0|   N|                Y|
|          2|            0|          0|   N|                Y|
|          5|            1|          1|   N|                N|
|          5|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [6]:
# Filter for greater than 20 total votes
total_votes_df = filtered_pet_products_df.filter(filtered_pet_products_df["total_votes"] >= 20)
total_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [7]:
# Filter for greater than 50% helpful vote percentage
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"] >= 0.5)
helpful_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



**Describe the statistics**

In [8]:
# Paid reviews
paid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'Y')
paid_df.describe().show(10)

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|               170|               170|               170| 170|              170|
|   mean| 3.929411764705882|63.641176470588235| 68.03529411764706|null|             null|
| stddev|1.1124708391270806|107.53304464635703|110.81862649267784|null|             null|
|    min|                 1|                13|                20|   Y|                N|
|    max|                 5|              1239|              1277|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



In [9]:
 # Unpaid reviews
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'N')
unpaid_df.describe().show()

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             37840|             37840|             37840|37840|            37840|
|   mean|3.7647463002114163| 57.50052854122622|61.863583509513745| null|             null|
| stddev|1.6055561346315699|107.60541722886974|111.92734632290404| null|             null|
|    min|                 1|                10|                20|    N|                N|
|    max|                 5|              5735|              5811|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



**Determine the percentage of five star review among vine reviews**

In [10]:
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = round(float(paid_five_star_number) / float(paid_number),4)
print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Number of paid reviews 170
Number of paid five star reviews 65
Percantage of paid reviews that are five stars 38.24%


**Determine the percentage of five-star review amaong non Vine reviews**

In [11]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = round(float(unpaid_five_star_number) / float(unpaid_number),4)
print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 37840
Number of unpaid five star reviews 20612
Percantage of paid reviews that are five stars 54.47%


**Summary of findings and analysis:**

1. There were 170 pet product reviews and 37840 unpaid reviews.
2. The number of paid five star reviews were 65 and unpaid five star reviews were 20612.
From the above information, we can conclude that we can rely on the Vine reviews as the five star rating for unpaid reviews by far outweighs the paid reviews by almost 16%.